In [2]:
import cv2
import numpy as np
import tensorflow as tf

# Load the pre-trained face detection model (Haar Cascade)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Load the trained gender detection model
model = tf.keras.models.load_model('gender_detection_trained_model.h5')

# Set image size (consistent with what the model was trained on)
img_height, img_width = 180, 180

# Class names for prediction
class_names = ['men', 'women']

# Function to preprocess the face image before prediction
def preprocess_face(face):
    face = cv2.resize(face, (img_height, img_width))
    face = np.expand_dims(face, axis=0)  # Create a batch
    return face / 255.0  # Normalize

# Start the webcam
cap = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    if not ret:
        break

    # Convert frame to grayscale (needed for face detection)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Loop over the faces
    for (x, y, w, h) in faces:
        # Draw a rectangle around the face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Extract the face region of interest (ROI)
        face_roi = frame[y:y+h, x:x+w]

        # Preprocess the face for the model
        preprocessed_face = preprocess_face(face_roi)

        # Make the gender prediction
        predictions = model.predict(preprocessed_face)
        predicted_class = np.argmax(predictions, axis=1)

        # Map predicted class to label
        gender = class_names[predicted_class[0]]

        # Display the gender prediction on the frame
        cv2.putText(frame, gender, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Display the resulting frame with bounding boxes and gender
    cv2.imshow('Gender Detection with Bounding Box', frame)

    # Break the loop on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()
